# Production comparison: NCAR CCSM4 RCP85

This notebook is for evaluating the results from the comparison of newly restacked files with existing production files.

It is meant to serve as a historical record and will not maintain functionality as files are moved.

Set up the environment:

In [1]:
import importlib.util
import os
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import xarray as xr


# all this code to load the config and luts modules by absolute path
project_dir = Path(os.getenv("PROJECT_DIR"))

def load_module(path):
    """https://docs.python.org/3/library/importlib.html#importing-a-source-file-directly"""
    module = path.name.split(".py")[0]
    spec = importlib.util.spec_from_file_location(
        module, path
    )
    module_obj = importlib.util.module_from_spec(spec)
    sys.modules[module] = module_obj
    spec.loader.exec_module(module_obj)
    
    return module_obj

luts = load_module(project_dir.joinpath("restack_20km/luts.py"))
config = load_module(project_dir.joinpath("restack_20km/config.py"))

## Hourly data

Load the results from comparing hourly data:

In [70]:
hourly_fp = config.anc_dir.joinpath(
    "production_data_comparisons",
    f"prod_comparison_{luts.groups[config.group]['fn_str']}_hourly.csv"
)
hourly_df = pd.read_csv(hourly_fp, parse_dates=["timestamp"])

scratch_dir = Path(hourly_df.iloc[0]["scratch_filename"]).parent.parent
prod_dir = Path(hourly_df.iloc[0]["prod_filename"]).parent.parent

#### Timestamp mismatches

Okay looks like we have some missing raw data! Swell...

We expect time mismatches with the newly rotated wind data created in 2021. Those wind data time stamps were labeled incorrectly (should not have leap days). 
However, for all variables (including wind) for the year 2022, the time mismatch is due to some missing WRF files for 2022-10-15, hours 12 and 13:

In [3]:
ls /archive/DYNDOWN/DIONE/pbieniek/ccsm/rcp85/hourly/2022/*2022-10-25_1*

/archive/DYNDOWN/DIONE/pbieniek/ccsm/rcp85/hourly/2022/WRFDS_d01.2022-10-25_10.nc*
/archive/DYNDOWN/DIONE/pbieniek/ccsm/rcp85/hourly/2022/WRFDS_d01.2022-10-25_11.nc*
/archive/DYNDOWN/DIONE/pbieniek/ccsm/rcp85/hourly/2022/WRFDS_d01.2022-10-25_14.nc*
/archive/DYNDOWN/DIONE/pbieniek/ccsm/rcp85/hourly/2022/WRFDS_d01.2022-10-25_15.nc*
/archive/DYNDOWN/DIONE/pbieniek/ccsm/rcp85/hourly/2022/WRFDS_d01.2022-10-25_16.nc*
/archive/DYNDOWN/DIONE/pbieniek/ccsm/rcp85/hourly/2022/WRFDS_d01.2022-10-25_17.nc*
/archive/DYNDOWN/DIONE/pbieniek/ccsm/rcp85/hourly/2022/WRFDS_d01.2022-10-25_18.nc*
/archive/DYNDOWN/DIONE/pbieniek/ccsm/rcp85/hourly/2022/WRFDS_d01.2022-10-25_19.nc*


So let's make sure that any time mistmatches for variables in non-leap years were all actually in 2022:

In [69]:
test = hourly_df.query("~timestamp.dt.is_leap_year & time_result == False & prod_exists == True")
test

,varname,scratch_filename,prod_filename,prod_exists,timestamp,arr_result,time_result,error
397,hfx,/import/SNAP/wrf_data/project_data/wrf_data/re...,/import/SNAP/wrf_data/project_data/wrf_data/ho...,True,2022-12-24 13:00:00,True,False,NaN
882,pcpnc,/import/SNAP/wrf_data/project_data/wrf_data/re...,/import/SNAP/wrf_data/project_data/wrf_data/ho...,True,2022-11-16 09:00:00,True,False,NaN
1738,swupb,/import/SNAP/wrf_data/project_data/wrf_data/re...,/import/SNAP/wrf_data/project_data/wrf_data/ho...,True,2022-11-26 03:00:00,True,False,NaN
2422,cldfra_mid,/import/SNAP/wrf_data/project_data/wrf_data/re...,/import/SNAP/wrf_data/project_data/wrf_data/ho...,True,2022-12-26 06:00:00,True,False,NaN
3363,smois,/import/SNAP/wrf_data/project_data/wrf_data/re...,/import/SNAP/wrf_data/project_data/wrf_data/ho...,True,2022-12-29 13:00:00,True,False,NaN
4241,ubot,/import/SNAP/wrf_data/project_data/wrf_data/re...,/import/SNAP/wrf_data/project_data/wrf_data/ho...,True,2022-12-14 17:00:00,True,False,NaN


Indeed they are. We are just going to correct this issue here and now. Maybe not the best spot, but at least it will be tracked in this notebook, which should not be updated.

Also, since there will be multiple issues to test here, start a running list of which rows have been addressed:

In [71]:
rows_addressed = []

### Patching in production data

We will want to iterate over all production files for 2022, and just patch the 12 and 13 hour timestamps from the production array into the new file.

In [5]:
def patch_scratch(scratch_fp, prod_fp, varname):
    # We want to keep everything but the time dimension in the final dataset
    scratch_ds = xr.open_dataset(scratch_fp)
    try:
        _ = scratch_ds.sel(time="2022-10-25 12:00:00")
    except KeyError:
        prod_ds = xr.open_dataset(prod_fp)
        new_scratch_ds = scratch_ds.drop_dims("time")
        prod_da_sl = prod_ds[varname].sel(time=slice("2022-10-25 12:00:00", "2022-10-25 13:00:00"))
        new_scratch_ds[varname] = xr.concat([scratch_ds[varname], prod_da_sl], dim="time").sortby("time")
        
        # cannot overwrite netcdf file so (permission error with xarray/netcdf4)
        #  locking the file or something so we need to save to different file and then rename.
        temp_scratch_fp = Path(str(scratch_fp).replace(".nc", "_tmp.nc"))
        new_scratch_ds.to_netcdf(temp_scratch_fp, engine="netcdf4")
        _ = temp_scratch_fp.rename(scratch_fp)

    return

In [6]:
%%time
import tqdm


for varname in tqdm.tqdm(luts.varnames):
    varname = varname.lower()
    scratch_fp = scratch_dir.joinpath(varname, f"{varname}_hourly_wrf_NCAR-CCSM4_rcp85_2022.nc")
    prod_fp = prod_dir.joinpath(varname, f"{varname}_hourly_wrf_NCAR-CCSM4_rcp85_2022.nc")
    patch_scratch(scratch_fp, prod_fp, varname)


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 46/46 [6:23:38<00:00, 500.41s/it]

CPU times: user 2h 36min 3s, sys: 2h 31min 30s, total: 5h 7min 33s
Wall time: 6h 23min 38s


Check one of the files and make sure the time slice is there:

In [8]:
scratch_dir = Path(hourly_df.iloc[0]["scratch_filename"]).parent.parent
scratch_fp = scratch_dir.joinpath("t2", "t2_hourly_wrf_NCAR-CCSM4_rcp85_2022.nc")
with xr.open_dataset(scratch_fp) as ds:
    print(ds["t2"].sel(time="2022-10-25 12:00:00"))

<xarray.DataArray 't2' (yc: 262, xc: 262)>
array([[242.11812, 242.25912, 242.41013, ..., 257.6151 , 257.67813, 257.73813],
       [241.25212, 241.85812, 242.24713, ..., 259.23914, 259.1111 , 257.37314],
       [240.62112, 241.11412, 241.18512, ..., 259.99112, 259.30112, 257.32413],
       ...,
       [289.78113, 289.68213, 289.62512, ..., 289.06512, 289.05414, 288.93112],
       [289.90213, 289.79913, 289.7441 , ..., 289.10312, 289.10812, 288.99213],
       [290.02112, 289.95514, 289.91312, ..., 289.00912, 289.03214, 289.05713]],
      dtype=float32)
Coordinates:
  * yc           (yc) float64 -1.824e+05 -2.024e+05 ... -5.382e+06 -5.402e+06
  * xc           (xc) float64 -2.61e+06 -2.59e+06 ... 2.59e+06 2.61e+06
    lon          (yc, xc) float32 ...
    lat          (yc, xc) float32 ...
    spatial_ref  int64 ...
    time         datetime64[ns] 2022-10-25T12:00:00
Attributes:
    initial_time:             12/30/2021 (18:00)
    forecast_time_units:      hours
    forecast_time:          

Okay so this should have taken care of the issue of mistmatch time stamp test cases for 2022. Consider these issues in the hourly dataframe taken care of:

In [72]:
rows_addressed.extend(test.index)

#### Wind leap year mismatches

We can now just check that the remaining mismatches are due to incorrectly (time-)indexed wind datasets during leap years. For example, look at dates before and after leap day in 2008:

In [19]:
prod_fp = "/import/SNAP/wrf_data/project_data/wrf_data/hourly_fix/u/u_hourly_wrf_NCAR-CCSM4_rcp85_2008.nc"
scratch_fp = "/import/SNAP/wrf_data/project_data/wrf_data/restacked/hourly/u/u_hourly_wrf_NCAR-CCSM4_rcp85_2008.nc"
scratch_ds = xr.open_dataset(scratch_fp)
prod_ds = xr.open_dataset(prod_fp)

Before leap day things match:

In [21]:
tstamp = "2008-01-24 21:00:00"
varname = "u"
assert np.all(scratch_ds[varname].sel(time=tstamp, plev=50) == prod_ds[varname].sel(time=tstamp, plev=50))

After leap day, they don't:

In [22]:
tstamp = "2008-03-24 21:00:00"
varname = "u"
assert np.all(scratch_ds[varname].sel(time=tstamp, plev=50) == prod_ds[varname].sel(time=tstamp, plev=50))

AssertionError: 

But we know these data should not have leap days. This is probably a bug that was introduced during the repair of the wind data. E.g. see a query for leap day from two production files, wind and non-wind. 

Wind:

In [23]:
prod_fp = config.base_dir.joinpath("hourly_fix/u/u_hourly_wrf_NCAR-CCSM4_rcp85_2008.nc")
prod_ds = xr.open_dataset(prod_fp)
_ = prod_ds["u"].sel(time="2008-02-29 01:00:00", plev=50)

And a non-wind:

In [24]:
fp = config.base_dir.joinpath("hourly_fix/t2/t2_hourly_wrf_NCAR-CCSM4_rcp85_2008.nc")
with xr.open_dataset(fp) as ds:
    try:
        _ = ds["t2"].sel(time="2008-02-29 01:00:00")
    except KeyError:
        _ = ds["t2"].sel(time="2008-02-28 01:00:00")
        print("No leap day exists")

No leap day exists


So this issue is only present in the wind data because of us having worked on it more recently, and introducing a bug in the restacking code used then. We will assume that all wind files for leap years that have mismatched arrays are due to this bug (which is now fixed). Verify that all such files were leap years:

In [74]:
test = hourly_df.query(
    "time_result == False & arr_result == False & varname in @wind_varnames"
).timestamp.dt.is_leap_year
try:
    assert np.all(test)
    rows_addressed.extend(test.index)
except AssertionError:
    print("Test failed!")

Now let's look at comparisons that resulted in errors. It looks like these are all files that don't exist because they come from the split year between historical and projected data, 2005. Confirm this:

In [75]:
test = hourly_df.query(
    "error == 'FileNotFoundError'"
).timestamp.dt.year == 2005

try:
    assert np.all(test)
    rows_addressed.extend(test.index)
except AssertionError:
    print("Test failed!")

And it looks like we had one Runtime error. This is probably a corrupt wind file:

In [51]:
print("Runtime errors: ", len(hourly_df.query("error == 'RuntimeError'")))
row = hourly_df.query("error == 'RuntimeError'").iloc[0]
print(row)
with xr.open_dataset(row["prod_filename"]) as prod_ds:
    with xr.open_dataset(row["scratch_filename"]) as scratch_ds:
        try:
            prod_ds[row["varname"]].sel(
                time=row["timestamp"]
            ).load()
        except RuntimeError:
            print("\nStill Runtime error")

Runtime errors:  1
varname                                                             u
scratch_filename    /import/SNAP/wrf_data/project_data/wrf_data/re...
prod_filename       /import/SNAP/wrf_data/project_data/wrf_data/ho...
prod_exists                                                      True
timestamp                                         2057-10-17 02:00:00
arr_result                                                      False
time_result                                                       NaN
error                                                    RuntimeError
Name: 3889, dtype: object

Still Runtime error


Track this row:

In [76]:
rows_addressed.extend([row.name])

Now tack the rows that were successful comparisons:

In [77]:
rows_addressed.extend(hourly_df.query("time_result == True & prod_exists == True").index)

And verify that we have checked every case in the results dataframe:

In [79]:
assert set(hourly_df.index) - set(rows_addressed) == set()

So the hourly data for NCAR CCSM4 RCP 8.5 passes the comparison with production data.

## Daily data

Load the results from comparing daily data:

In [81]:
daily_fp = config.anc_dir.joinpath(
    "production_data_comparisons",
    f"prod_comparison_{luts.groups[config.group]['fn_str']}_daily.csv"
)
daily_df = pd.read_csv(daily_fp, parse_dates=["timestamp"])

Okay looks like all of the daily files can be accounted for based on the array result comparison:

In [103]:
test = sorted(list(daily_df.query("arr_result == True").index) + list(daily_df.query("arr_result == False").index))
assert np.all(daily_df.index == test)

And it looks like all of the files that didn't match were due to the missing 2005 files:

In [102]:
assert np.all(daily_df.query("arr_result == False").error == "FileNotFoundError")

And that covers it for the daily data.